# Signal Chain Simulation

This notebook demonstrates the end-to-end radiometric signal chain in COMPASS. The signal
chain models how photons from a scene reach the sensor and are converted to electrons:

$$\text{Signal}_i = \int L(\lambda)\; R_{\text{scene}}(\lambda)\; T_{\text{optics}}(\lambda)\; QE_i(\lambda)\; d\lambda$$

where:
- $L(\lambda)$ is the illuminant spectral power distribution
- $R_{\text{scene}}(\lambda)$ is the scene reflectance
- $T_{\text{optics}}(\lambda)$ is the combined lens + IR filter transmittance
- $QE_i(\lambda)$ is the quantum efficiency of pixel channel $i$

We will cover:
1. Creating and comparing illuminants (D65, A, LED)
2. Defining scene reflectances (grey card, Macbeth patches)
3. Setting up smartphone module optics with IR cut filter
4. Creating synthetic QE curves for R/G/B pixels
5. Computing spectral irradiance at each stage
6. Integrating signal per channel under each illuminant
7. Computing white balance gains
8. Estimating SNR for different exposure times
9. Comparing color ratios across illuminants

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from compass.sources.illuminant import Illuminant
from compass.sources.scene import SceneReflectance
from compass.optics.module_optics import ModuleOptics, IRCutFilter, LensTransmittance
from compass.analysis.signal_calculator import SignalCalculator
from compass.core.units import um_to_nm

In [ ]:
# Common wavelength grid for the entire signal chain (380-780 nm, 1 nm steps)
wavelengths = np.linspace(0.38, 0.78, 401)  # um
wl_nm = um_to_nm(wavelengths)                # nm, for plotting

## 1. Illuminants

An illuminant describes the spectral power distribution (SPD) of a light source.
COMPASS provides factory methods for common illuminants:

- **CIE D65**: average daylight (~6504 K), the standard for sRGB white point.
- **CIE A**: incandescent tungsten (2856 K), warm yellowish light.
- **LED white**: phosphor-converted white LED at a specified CCT.

Let us create all three and compare their spectral shapes.

In [ ]:
# Create illuminants
ill_d65 = Illuminant.cie_d65(wavelengths)
ill_a = Illuminant.cie_a(wavelengths)
ill_led = Illuminant.led_white(5000.0, wavelengths)

# Plot
fig, ax = plt.subplots(figsize=(9, 5))
ax.plot(wl_nm, ill_d65.spectrum, color="tab:blue", linewidth=2, label="CIE D65 (daylight)")
ax.plot(wl_nm, ill_a.spectrum, color="tab:orange", linewidth=2, label="CIE A (incandescent)")
ax.plot(wl_nm, ill_led.spectrum, color="tab:green", linewidth=2, label="LED 5000K")

ax.set_xlabel("Wavelength (nm)", fontsize=12)
ax.set_ylabel("Relative SPD", fontsize=12)
ax.set_title("Illuminant Spectral Power Distributions", fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim(380, 780)
ax.set_ylim(0, 1.15)
fig.tight_layout()
plt.show()

print(f"Illuminant names: {ill_d65.name}, {ill_a.name}, {ill_led.name}")

## 2. Scene Reflectances

Scene reflectance $R(\lambda)$ describes how much light a surface reflects at each wavelength.
COMPASS provides:

- `SceneReflectance.flat(value, wl)` -- uniform reflectance (e.g., 18% grey card).
- `SceneReflectance.macbeth_patch(id, wl)` -- Macbeth ColorChecker approximations.
- `SceneReflectance.color_target(color, wl)` -- simple colour targets.

We will use an 18% grey card and three Macbeth patches: red (#15), green (#14),
and blue (#13).

In [ ]:
# Create scene reflectances
grey_card = SceneReflectance.flat(0.18, wavelengths)
macbeth_red = SceneReflectance.macbeth_patch(15, wavelengths)    # Red
macbeth_green = SceneReflectance.macbeth_patch(14, wavelengths)  # Green
macbeth_blue = SceneReflectance.macbeth_patch(13, wavelengths)   # Blue

# Plot
fig, ax = plt.subplots(figsize=(9, 5))
ax.plot(wl_nm, grey_card.reflectance, color="grey", linewidth=2, label="18% Grey Card")
ax.plot(wl_nm, macbeth_red.reflectance, color="red", linewidth=2, label=f"Macbeth Red (#{15})")
ax.plot(wl_nm, macbeth_green.reflectance, color="green", linewidth=2, label=f"Macbeth Green (#{14})")
ax.plot(wl_nm, macbeth_blue.reflectance, color="blue", linewidth=2, label=f"Macbeth Blue (#{13})")

ax.set_xlabel("Wavelength (nm)", fontsize=12)
ax.set_ylabel("Reflectance", fontsize=12)
ax.set_title("Scene Reflectance Spectra", fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim(380, 780)
ax.set_ylim(0, 1.0)
fig.tight_layout()
plt.show()

## 3. Module Optics

The `ModuleOptics` class combines a multi-element lens transmittance model and an IR
cut-off filter. A typical smartphone camera module has:

- 6 plastic lens elements with AR coatings
- An IR cut filter at ~650 nm to block near-infrared light

We will create a smartphone module and visualise the individual and combined transmittances.

In [ ]:
# Create a smartphone module (6-element lens + IR cut at 650 nm)
module = ModuleOptics.smartphone_module()

# Also get the individual component transmittances
t_lens = module.lens.transmittance(wavelengths)
t_ir = module.ir_filter.transmittance(wavelengths)
t_total = module.total_transmittance(wavelengths)

# Plot
fig, ax = plt.subplots(figsize=(9, 5))
ax.plot(wl_nm, t_lens, color="tab:blue", linewidth=2, linestyle="--", label="Lens (6P, AR coated)")
ax.plot(wl_nm, t_ir, color="tab:red", linewidth=2, linestyle="--", label="IR Cut Filter")
ax.plot(wl_nm, t_total, color="black", linewidth=2.5, label="Combined Transmittance")

ax.set_xlabel("Wavelength (nm)", fontsize=12)
ax.set_ylabel("Transmittance", fontsize=12)
ax.set_title("Smartphone Camera Module Optics", fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim(380, 780)
ax.set_ylim(0, 1.05)
fig.tight_layout()
plt.show()

print(f"Peak transmittance: {np.max(t_total):.3f}")
print(f"Transmittance at 550 nm: {t_total[np.argmin(np.abs(wl_nm - 550))]:.3f}")
print(f"Transmittance at 700 nm: {t_total[np.argmin(np.abs(wl_nm - 700))]:.3f}")

## 4. Synthetic Quantum Efficiency Curves

The quantum efficiency QE(lam) represents the probability that a photon of wavelength
lam is converted to an electron in a given pixel. In a Bayer sensor, each pixel has a
colour-dependent QE shaped primarily by the colour filter absorption.

Here we create synthetic QE curves using Gaussian peaks for R, G, B channels. In a
full COMPASS simulation, these would be computed from the electromagnetic solver output.

In [ ]:
def gaussian_qe(wavelengths, peak_wl, sigma, peak_qe=0.7):
    """Create a Gaussian QE curve."""
    return peak_qe * np.exp(-0.5 * ((wavelengths - peak_wl) / sigma) ** 2)

# Synthetic QE: R peaks at 620 nm, G at 530 nm, B at 460 nm
qe_r = gaussian_qe(wavelengths, peak_wl=0.620, sigma=0.055, peak_qe=0.65)
qe_g = gaussian_qe(wavelengths, peak_wl=0.530, sigma=0.050, peak_qe=0.70)
qe_b = gaussian_qe(wavelengths, peak_wl=0.460, sigma=0.040, peak_qe=0.55)

# Package into the dict format expected by SignalCalculator
qe_per_pixel = {
    "R": qe_r,
    "G": qe_g,
    "B": qe_b,
}

# Plot
fig, ax = plt.subplots(figsize=(9, 5))
ax.plot(wl_nm, qe_r, color="red", linewidth=2, label="Red QE")
ax.plot(wl_nm, qe_g, color="green", linewidth=2, label="Green QE")
ax.plot(wl_nm, qe_b, color="blue", linewidth=2, label="Blue QE")

ax.set_xlabel("Wavelength (nm)", fontsize=12)
ax.set_ylabel("Quantum Efficiency", fontsize=12)
ax.set_title("Synthetic Pixel QE Curves", fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim(380, 780)
ax.set_ylim(0, 0.85)
fig.tight_layout()
plt.show()

## 5. Spectral Irradiance Cascade

Now we combine all the pieces. The spectral irradiance at the sensor plane is:

$$E(\lambda) = L(\lambda) \cdot R(\lambda) \cdot T(\lambda)$$

and the signal in each pixel channel is:

$$S_i(\lambda) = E(\lambda) \cdot QE_i(\lambda)$$

Let us plot the cascade for D65 illumination of the 18% grey card.

In [ ]:
calc = SignalCalculator(wavelengths)

# Spectral irradiance at sensor: L(lam) * R(lam) * T(lam)
E_sensor = calc.compute_spectral_irradiance(ill_d65, grey_card, module)

# Signal per channel: E(lam) * QE_i(lam)
S_r = E_sensor * qe_r
S_g = E_sensor * qe_g
S_b = E_sensor * qe_b

fig, axes = plt.subplots(2, 2, figsize=(13, 9))

# Panel 1: Illuminant x Reflectance
ax = axes[0, 0]
l_interp = ill_d65.interpolate(wavelengths)
r_interp = grey_card.interpolate(wavelengths)
ax.fill_between(wl_nm, l_interp, alpha=0.3, color="gold", label="D65 SPD")
ax.plot(wl_nm, l_interp * r_interp, color="tab:brown", linewidth=2,
        label=r"D65 $\times$ Grey Card")
ax.set_xlabel("Wavelength (nm)")
ax.set_ylabel("Relative Intensity")
ax.set_title("Stage 1: Illuminant x Scene Reflectance")
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 2: After module optics
ax = axes[0, 1]
ax.plot(wl_nm, l_interp * r_interp, color="tab:brown", linewidth=1.5, alpha=0.5,
        linestyle="--", label="Before optics")
ax.plot(wl_nm, E_sensor, color="black", linewidth=2, label="After optics")
ax.set_xlabel("Wavelength (nm)")
ax.set_ylabel("Spectral Irradiance (rel.)")
ax.set_title("Stage 2: After Module Optics")
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 3: Per-channel signal spectra
ax = axes[1, 0]
ax.fill_between(wl_nm, S_r, alpha=0.4, color="red", label="Red")
ax.fill_between(wl_nm, S_g, alpha=0.4, color="green", label="Green")
ax.fill_between(wl_nm, S_b, alpha=0.4, color="blue", label="Blue")
ax.set_xlabel("Wavelength (nm)")
ax.set_ylabel(r"$E(\lambda) \times QE_i(\lambda)$")
ax.set_title("Stage 3: Per-Channel Signal Spectra")
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 4: Integrated signal bars
ax = axes[1, 1]
signals = calc.compute_pixel_signal(ill_d65, grey_card, module, qe_per_pixel)
channels = list(signals.keys())
values = [signals[ch] for ch in channels]
bar_colors = {"R": "red", "G": "green", "B": "blue"}
ax.bar(channels, values, color=[bar_colors.get(ch, "grey") for ch in channels],
       edgecolor="black", alpha=0.7)
ax.set_xlabel("Channel")
ax.set_ylabel("Integrated Signal (rel.)")
ax.set_title("Stage 4: Integrated Signal per Channel")
ax.grid(True, alpha=0.3, axis="y")

fig.suptitle("Signal Chain Cascade: D65 + 18% Grey Card", fontsize=15, y=1.01)
fig.tight_layout()
plt.show()

## 6. Signal Under Multiple Illuminants

The signal level and color balance change significantly depending on the illuminant.
Incandescent light (CIE A) has much more red energy than daylight, which shifts the
R/G/B signal ratios. Let us compute the integrated signal per channel for each
illuminant viewing the grey card.

In [ ]:
illuminants = {
    "D65 (daylight)": ill_d65,
    "A (incandescent)": ill_a,
    "LED 5000K": ill_led,
}

# Compute signals for each illuminant
all_signals = {}
for ill_name, ill in illuminants.items():
    sig = calc.compute_pixel_signal(ill, grey_card, module, qe_per_pixel)
    all_signals[ill_name] = sig
    print(f"{ill_name:>25s}: R={sig['R']:.6f}  G={sig['G']:.6f}  B={sig['B']:.6f}")

# Grouped bar chart
fig, ax = plt.subplots(figsize=(9, 5))
x = np.arange(len(illuminants))
width = 0.22
ill_names = list(illuminants.keys())

for i, (ch, color) in enumerate([("R", "red"), ("G", "green"), ("B", "blue")]):
    values = [all_signals[name][ch] for name in ill_names]
    ax.bar(x + (i - 1) * width, values, width, color=color, alpha=0.7,
           edgecolor="black", label=f"{ch} channel")

ax.set_xlabel("Illuminant", fontsize=12)
ax.set_ylabel("Integrated Signal (rel.)", fontsize=12)
ax.set_title("Signal per Channel under Different Illuminants", fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(ill_names)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis="y")
fig.tight_layout()
plt.show()

## 7. White Balance Gains

White balance corrects for the colour cast caused by different illuminants. The algorithm
computes gains that make R, G, B channels equal for a neutral grey target under a given
illuminant. The green channel gain is normalised to 1.0.

$$\text{gain}_i = \frac{S_G}{S_i}$$

In [ ]:
# We use the SignalCalculator.white_balance_gains method, which needs a bayer_map.
# Since we have simple R/G/B pixels (not a full Bayer grid), we define a minimal map.
bayer_map = [["R", "G"], ["G", "B"]]

print(f"{'Illuminant':>25s}  {'R gain':>8s}  {'G gain':>8s}  {'B gain':>8s}")
print("-" * 58)

wb_gains_all = {}
for ill_name, ill in illuminants.items():
    gains = calc.white_balance_gains(ill, module, qe_per_pixel, bayer_map)
    wb_gains_all[ill_name] = gains
    print(f"{ill_name:>25s}  {gains.get('R', 0):>8.4f}  {gains.get('G', 0):>8.4f}  {gains.get('B', 0):>8.4f}")

# Visualize gains
fig, ax = plt.subplots(figsize=(9, 5))
x = np.arange(len(illuminants))
width = 0.22
ill_names = list(illuminants.keys())

for i, (ch, color) in enumerate([("R", "red"), ("G", "green"), ("B", "blue")]):
    values = [wb_gains_all[name].get(ch, 1.0) for name in ill_names]
    ax.bar(x + (i - 1) * width, values, width, color=color, alpha=0.7,
           edgecolor="black", label=f"{ch} gain")

ax.axhline(1.0, color="grey", linestyle="--", alpha=0.5)
ax.set_xlabel("Illuminant", fontsize=12)
ax.set_ylabel("White Balance Gain", fontsize=12)
ax.set_title("White Balance Gains (G = 1.0)", fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(ill_names)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis="y")
fig.tight_layout()
plt.show()

## 8. SNR vs Exposure Time

The signal-to-noise ratio (SNR) depends on the number of collected electrons:

$$\text{SNR} = \frac{N_{\text{signal}}}{\sqrt{N_{\text{signal}} + N_{\text{dark}} + N_{\text{read}}^2}}$$

We estimate electron counts using the `SignalCalculator.compute_signal_electrons` method,
then compute SNR for a range of exposure times.

Parameters: 1.0 um pixel (area = 1.0 um^2), f/2.0 lens, read noise = 2 e-, dark current = 0.1 e/s.

In [ ]:
# Camera parameters
pixel_area = 1.0     # um^2 (1.0 um pitch)
f_number = 2.0
read_noise = 2.0     # e- rms
dark_current = 0.1   # e-/s

exposure_times = np.logspace(-4, -1, 30)  # 0.1 ms to 100 ms

# Compute relative pixel signals for D65 + grey card
pixel_signals = calc.compute_pixel_signal(ill_d65, grey_card, module, qe_per_pixel)

# Sweep exposure time and compute SNR for each channel
snr_vs_time = {ch: [] for ch in ["R", "G", "B"]}
electrons_vs_time = {ch: [] for ch in ["R", "G", "B"]}

for t_exp in exposure_times:
    electrons = calc.compute_signal_electrons(
        pixel_signals, exposure_time=t_exp,
        pixel_area=pixel_area, f_number=f_number,
    )
    snr = calc.compute_snr(
        electrons, read_noise=read_noise,
        dark_current=dark_current, exposure_time=t_exp,
    )
    for ch in ["R", "G", "B"]:
        snr_vs_time[ch].append(snr[ch])
        electrons_vs_time[ch].append(electrons[ch])

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

for ch, color in [("R", "red"), ("G", "green"), ("B", "blue")]:
    ax1.plot(exposure_times * 1000, electrons_vs_time[ch], color=color,
             linewidth=2, label=ch)
    ax2.plot(exposure_times * 1000, snr_vs_time[ch], color=color,
             linewidth=2, label=ch)

ax1.set_xlabel("Exposure Time (ms)", fontsize=12)
ax1.set_ylabel("Signal Electrons", fontsize=12)
ax1.set_title("Electron Count vs Exposure Time", fontsize=14)
ax1.set_xscale("log")
ax1.set_yscale("log")
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

ax2.set_xlabel("Exposure Time (ms)", fontsize=12)
ax2.set_ylabel("SNR", fontsize=12)
ax2.set_title("SNR vs Exposure Time (D65, 18% grey)", fontsize=14)
ax2.set_xscale("log")
ax2.set_yscale("log")
# Reference SNR lines
ax2.axhline(20, color="grey", linestyle=":", alpha=0.4)
ax2.text(0.15, 22, "SNR = 20 dB", fontsize=9, color="grey")
ax2.axhline(40, color="grey", linestyle=":", alpha=0.4)
ax2.text(0.15, 44, "SNR = 40", fontsize=9, color="grey")
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

fig.suptitle("D65 Illuminant, 18% Grey Card, f/2.0, 1.0 um pixel", fontsize=13, y=1.01)
fig.tight_layout()
plt.show()

# Print SNR at a typical exposure
t_typical = 1.0 / 60.0  # 1/60 s
elec = calc.compute_signal_electrons(
    pixel_signals, exposure_time=t_typical,
    pixel_area=pixel_area, f_number=f_number,
)
snr_typical = calc.compute_snr(
    elec, read_noise=read_noise,
    dark_current=dark_current, exposure_time=t_typical,
)
print(f"\nAt t_exp = {t_typical*1000:.1f} ms (1/60 s):")
for ch in ["R", "G", "B"]:
    print(f"  {ch}: {elec[ch]:.0f} e-,  SNR = {snr_typical[ch]:.1f}")

## 9. Color Ratios Across Illuminants

Color fidelity depends on how consistently the sensor reproduces colour under different
lighting. We compute R/G and B/G ratios for each illuminant viewing the grey card.
For a perfect grey under perfect white balance, both ratios should equal 1.0. Deviations
indicate the sensor's colour sensitivity to illuminant changes.

We also compute ratios for the coloured Macbeth patches to assess colour accuracy.

In [ ]:
# Color ratios for grey card under each illuminant
print("Color ratios for 18% grey card:")
print(f"{'Illuminant':>25s}  {'R/G':>8s}  {'B/G':>8s}")
print("-" * 45)

ratio_data = {}
for ill_name, ill in illuminants.items():
    sig = calc.compute_pixel_signal(ill, grey_card, module, qe_per_pixel)
    ratios = SignalCalculator.color_ratio(sig)
    ratio_data[ill_name] = ratios
    print(f"{ill_name:>25s}  {ratios['R/G']:>8.4f}  {ratios['B/G']:>8.4f}")

In [ ]:
# Color ratios for coloured patches under D65
scene_patches = {
    "18% Grey": grey_card,
    "Macbeth Red": macbeth_red,
    "Macbeth Green": macbeth_green,
    "Macbeth Blue": macbeth_blue,
}

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

for ill_name, ill, marker in [
    ("D65", ill_d65, "o"),
    ("A", ill_a, "s"),
    ("LED 5000K", ill_led, "^"),
]:
    rg_vals, bg_vals, labels = [], [], []
    for patch_name, patch in scene_patches.items():
        sig = calc.compute_pixel_signal(ill, patch, module, qe_per_pixel)
        ratios = SignalCalculator.color_ratio(sig)
        rg_vals.append(ratios["R/G"])
        bg_vals.append(ratios["B/G"])
        labels.append(patch_name)

    ax1.scatter(labels, rg_vals, s=80, marker=marker, label=ill_name, alpha=0.8)
    ax2.scatter(labels, bg_vals, s=80, marker=marker, label=ill_name, alpha=0.8)

ax1.set_ylabel("R/G Ratio", fontsize=12)
ax1.set_title("R/G Ratio by Scene Patch and Illuminant", fontsize=13)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3, axis="y")
ax1.tick_params(axis="x", rotation=15)

ax2.set_ylabel("B/G Ratio", fontsize=12)
ax2.set_title("B/G Ratio by Scene Patch and Illuminant", fontsize=13)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3, axis="y")
ax2.tick_params(axis="x", rotation=15)

fig.tight_layout()
plt.show()

# Determine which illuminant gives the most balanced grey-card response
print("\nGrey-card balance metric (|R/G - 1| + |B/G - 1|):")
best_ill = None
best_metric = np.inf
for ill_name in illuminants:
    rg = ratio_data[ill_name]["R/G"]
    bg = ratio_data[ill_name]["B/G"]
    metric = abs(rg - 1.0) + abs(bg - 1.0)
    print(f"  {ill_name:>25s}: {metric:.4f}")
    if metric < best_metric:
        best_metric = metric
        best_ill = ill_name

print(f"\nBest balance: {best_ill} (metric = {best_metric:.4f})")
print("A lower metric means the illuminant + sensor combination produces")
print("colour ratios closer to unity for a neutral target, requiring less")
print("white-balance correction.")

## Summary and Recommendations

This notebook demonstrated the full COMPASS radiometric signal chain:

| Stage | Component | Key class |
|-------|-----------|----------|
| Light source | Illuminant SPD | `Illuminant.cie_d65()`, `.cie_a()`, `.led_white()` |
| Scene | Reflectance spectrum | `SceneReflectance.flat()`, `.macbeth_patch()` |
| Optics | Lens + IR filter | `ModuleOptics.smartphone_module()` |
| Sensor | Pixel QE per channel | `SignalCalculator.compute_pixel_signal()` |
| Analysis | Electrons, SNR, WB | `compute_signal_electrons()`, `compute_snr()`, `white_balance_gains()` |
| Colour | R/G, B/G ratios | `SignalCalculator.color_ratio()` |

### Key findings

- **Illuminant A** shifts energy heavily toward red, requiring large blue-channel gain
  for white balance -- this amplifies blue-channel noise.
- **D65** and **LED 5000K** provide more balanced R/G/B signals, requiring less
  aggressive white balance correction.
- **SNR** is photon-shot-noise limited at moderate exposure times. At very short exposures,
  read noise becomes dominant.
- **Colour ratios** vary significantly across illuminants, motivating the need for robust
  auto white balance (AWB) algorithms.

### Extending this analysis

- Replace the synthetic Gaussian QE curves with simulation-derived QE from the RCWA/FDTD
  solvers in COMPASS.
- Add more scene patches (full 24-patch Macbeth chart) for a comprehensive colour
  accuracy analysis.
- Sweep pixel pitch and silicon thickness to study their impact on QE and SNR.
- Investigate the effect of chief ray angle (CRA) on colour uniformity across the sensor.